In [1]:
import socket
from Games import Go, Ataxx
from AlphaZero import AlphaZeroPlayer, CNNET
from training import args

host = '192.168.1.101'
port = 9000

In [2]:
class PlayGoGame:
    def __init__(self, board_size, client_socket, my_turn):
            self.game = Go(board_size,board_size)
            model = CNNET(self.game, args)
            model.load_model(f"./best_model/{self.game.model_name}/model.tar")
            self.player = AlphaZeroPlayer(model)
            self.client_socket = client_socket
            self.my_turn = my_turn            
            
    def apply_move(self, message):
        if message == "PASS":
            move = self.game.getActionSize() - 1
            self.game.apply_action(move)
        else:
            a,b = map(int, message.replace('MOVE ', '').split(','))
            move = (a,b)
            self.game.apply_action(self.game.encode(move))
        self.my_turn = not self.my_turn
        
    def receive_message(self):
        # Wait for server response
        response = self.client_socket.recv(1024).decode()
        return response

        
    def game_cycle(self):
        while True:
            if self.my_turn:
                action = self.player.get_action(self.game) 
                
                if action == self.game.getActionSize() - 1:
                    message = "PASS"
                else:
                    move = self.game.decode(action)
                    message = f"MOVE {str(move[0])},{str(move[1])}"
                
                self.client_socket.send(message.encode())
                print("Send:", message)
                
                response = self.receive_message()
                print(f"Server Response1: {response}")
                
                if "END" in response:
                    print("Game ended!")
                    break
                if response == "VALID".replace(" ", ""):
                    self.apply_move(message)
                else:
                    print("Invalid move!")
            else:
                response = self.receive_message()
                response=response[:8]
                print(f"Server Response 2: {response}")
                if "END" in response:
                    print("Game ended!")
                    break
                elif 'MOVE' in response:
                    self.apply_move(response)
                elif 'TIMEOUT' in response or 'PASS' in response:
                    self.apply_move("PASS")

In [3]:
class PlayAtaxxGame:
    def __init__(self, board_size, client_socket, my_turn):
        self.game = Ataxx(board_size,board_size)
        model = CNNET(self.game, args)
        model.load_model(f"./best_model/{self.game.model_name}/model.tar")
        self.player = AlphaZeroPlayer(model)
        self.client_socket = client_socket
        self.my_turn = my_turn
        
    def apply_move(self, message):
        a, b, c, d = map(int, message.replace('MOVE ', '').split(','))
        move = ((a,b), (c,d))
        self.game.apply_action(self.game.encode(move[0], move[1]))
        self.my_turn = not self.my_turn
            
    def receive_message(self):
        # Wait for server response
        response = self.client_socket.recv(1024).decode()
        return response
        
    def game_cycle(self):
        while True:
            if self.my_turn:
                move = self.game.decode(self.player.get_action(self.game))
                message = f"MOVE {str(move[0][0])}, {str(move[0][1])}, {str(move[1][0])}, {str(move[1][1])}"
                
                self.client_socket.send(message.encode())
                print("Send:", message)
                
                response = self.receive_message()
                print(f"Server Response1: {response}")
                
                if "END" in response:
                    print("Game ended!")
                    break
                if response == "VALID".replace(" ", ""):
                    self.apply_move(message)
                else:
                    print("Invalid move!")
            else:
                response = self.receive_message()
                print(f"Server Response 2: {response}")
                if "END" in response:
                    print("Game ended!")
                    break
                elif 'MOVE' in response:
                    self.apply_move(response)
                elif 'TIMEOUT' in response:
                    self.apply_move(response[8:])

In [4]:
def connect_to_server():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))

    response = client_socket.recv(1024).decode()
    print(f"Server Response INIT: {response}")

    game = response[-4:]
    print("Playing:", game)
    
    if "1" in response:
        my_turn = True
    else:
        my_turn = False

    if "G" in game:
        play_game = PlayGoGame(int(game[1]), client_socket, my_turn)
        play_game.game_cycle()
        
    if "A" in game:
        play_game = PlayAtaxxGame(int(game[1]), client_socket, my_turn)
        play_game.game_cycle()

In [5]:
if __name__ == "__main__":
    connect_to_server()

Server Response INIT: AG2 A6x6
Playing: A6x6
Server Response 2: MOVE 5, 5, 4, 5
Send: MOVE 0, 5, 2, 5
Server Response1: VALID
Server Response 2: MOVE 4, 5, 3, 4
Send: MOVE 5, 0, 3, 0
Server Response1: VALID
Server Response 2: MOVE 0, 0, 2, 1
Send: MOVE 0, 0, 0, 0


/Users/seb/Library/CloudStorage/OneDrive-UniversidadedoPorto/3ºY/LabIACD/AlphaZero/MonteCarlo.py:70: RuntimeWarning: invalid value encountered in divide
  policy /= np.sum(policy)
/Users/seb/Library/CloudStorage/OneDrive-UniversidadedoPorto/3ºY/LabIACD/AlphaZero/MonteCarlo.py:90: RuntimeWarning: invalid value encountered in divide
  action_probs /= np.sum(action_probs)


Server Response1: END
Game ended!
